In [5]:
import os
path_data = '/home/asap7772/asap7772/real_data_kitchen/bridge_data'
fd = os.listdir(path_data)
fd = [os.path.join(path_data, x) for x in fd]
print(fd)

['/home/asap7772/asap7772/real_data_kitchen/bridge_data/toykitchen1', '/home/asap7772/asap7772/real_data_kitchen/bridge_data/toysink2_bww', '/home/asap7772/asap7772/real_data_kitchen/bridge_data/toykitchen2_room8052', '/home/asap7772/asap7772/real_data_kitchen/bridge_data/toysink3_bww', '/home/asap7772/asap7772/real_data_kitchen/bridge_data/toysink1_room8052']


In [6]:
def deldir(top):
    import os
    for root, dirs, files in os.walk(top, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir(top)

In [7]:
from collections import defaultdict
from PIL import Image
from torchvision import transforms
import pickle
import numpy as np

def center_crop(path): #center crop from 480x640 to 480x480 and then resized to 64x64 and flattened as a tensor
    im = Image.open(path)

    width, height = im.size   # Get dimensions
    new_width, new_height = 480,480

    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    # Crop the center of the image
    im = im.crop((left, top, right, bottom))
    im = im.resize((64,64), Image.ANTIALIAS)
    trans = transforms.ToTensor()

    return trans(im).numpy().flatten()

def process_images(path): #processes images at a trajectory level
    names = sorted([x for x in os.listdir(path) if 'images' in x], key = lambda x: int(x.split('images')[1]))
    image_path = [os.path.join(path, x) for x in os.listdir(path) if 'images' in x]
    image_path = sorted(image_path, key = lambda x: int(x.split('images')[1]))

    images_out = defaultdict(list)
    
    get_num = lambda x: int(x.split('_')[1].split('.')[0])

    for i in range(len(names)):
        for im_name in sorted(os.listdir(image_path[i]), key=get_num):
            images_out[names[i]].append(center_crop(os.path.join(image_path[i], im_name)))

    images_out = dict(images_out)

    obs, next_obs = dict(), dict()

    for n in names:
        obs[n] = images_out[n][:-1]
        next_obs[n] = images_out[n][1:]
    return obs, next_obs

def process_state(path):
    fp = os.path.join(path, 'obs_dict.pkl')
    x = pickle.load(open(fp, 'rb'))  
    return x['full_state'][:-1], x['full_state'][1:]

def process_actions(path): # gets actions
    fp = os.path.join(path, 'policy_out.pkl')
    act_list = pickle.load(open(fp, 'rb'))
    return [x['actions'] for x in act_list]    

def process_dc(path): #processes each data collection attempt
    # print(path)
    all_dicts = list()
    for t in sorted(os.listdir(os.path.join(path, 'raw', 'traj_group0')), key = lambda x: int(x.split('traj')[1])):
        out = dict()
        tp = os.path.join(path, 'raw', 'traj_group0', t)
        
        ld = os.listdir(tp)

        assert 'obs_dict.pkl' in ld,  tp + ':' + str(ld) 
        assert 'policy_out.pkl' in  ld,  tp + ':' + str(ld)
        assert 'agent_data.pkl' in  ld,  tp + ':' + str(ld)
        
        obs, next_obs = process_images(tp)
        acts = process_actions(tp)
        state, next_state = process_state(tp)
        term = [0] * len(acts)
        
        out['observations'] = obs
        out['observations']['state'] = state
        out['next_observations'] = next_obs
        out['next_observations']['state'] = state

        out['observations'] = [dict(zip(out['observations'],t)) for t in zip(*out['observations'].values())]
        out['next_observations'] = [dict(zip(out['next_observations'],t)) for t in zip(*out['next_observations'].values())]
        
        out['actions'] = acts
        out['terminals'] = term
        
        all_dicts.append(out)
    return all_dicts

def make_numpy(path, outpath): # overarching npy creation
    lst = list()
    print(path)
    for dc in os.listdir(path):
        curr = process_dc(os.path.join(path,dc))
        lst.extend(curr)
    np.save(os.path.join(outpath, 'out.npy'), lst)
    print('saved', os.path.join(outpath, 'out.npy'))

In [8]:
conv_string = lambda x: x.split('bridge_data')[0] + 'bridge_data_numpy' + x.split('bridge_data')[1]
for p in fd:
    for t in os.listdir(p):
        tp = os.path.join(p,t)
        make_numpy(tp, conv_string(tp))

/home/asap7772/asap7772/real_data_kitchen/bridge_data/toykitchen1/put_lid_on_stove
saved /home/asap7772/asap7772/real_data_kitchen/bridge_data_numpy/toykitchen1/put_lid_on_stove/out.npy
/home/asap7772/asap7772/real_data_kitchen/bridge_data/toykitchen1/pick_up_pot_from_sink_distractors
saved /home/asap7772/asap7772/real_data_kitchen/bridge_data_numpy/toykitchen1/pick_up_pot_from_sink_distractors/out.npy
/home/asap7772/asap7772/real_data_kitchen/bridge_data/toykitchen1/put_sweet_potato_in_pan_which_is_on_stove
saved /home/asap7772/asap7772/real_data_kitchen/bridge_data_numpy/toykitchen1/put_sweet_potato_in_pan_which_is_on_stove/out.npy
/home/asap7772/asap7772/real_data_kitchen/bridge_data/toykitchen1/put_carrot_on_plate
saved /home/asap7772/asap7772/real_data_kitchen/bridge_data_numpy/toykitchen1/put_carrot_on_plate/out.npy
/home/asap7772/asap7772/real_data_kitchen/bridge_data/toykitchen1/put_carrot_on_cutting_board
saved /home/asap7772/asap7772/real_data_kitchen/bridge_data_numpy/toykit